In [ ]:
dev = qml.device('default.qubit', wires=1)
# Define the quantum circuit with a parameterized RY gate
@qml.qnode(dev)
def circuit_to_differentiate(theta):
    """Quantum circuit we want to differentiate.

    Args:
        theta (float): parameter of the circuit with respect to which we differentiate.

    Returns:
        (np.tensor): a numpy tensor of 1 element corresponding to the expectation value of Z
    """
    ##################
    # YOUR CODE HERE #
    ##################
    qml.RY(theta, wires=0)
    return qml.expval(qml.PauliZ(0))
# Define the parameter-shift rule function
def parameter_shift_rule(theta):
    """Function that applies the parameter shift rule to `circuit_to_differentiate` with respect to the parameter theta.

    Args:
        theta (float): parameter of the circuit with respect to which we differentiate.

    Returns:
        (np.tensor): a numpy tensor of 1 element corresponding to the derivative of the circuit with respect to theta.
    """    
    ##################
    # YOUR CODE HERE #
    ##################
    shift = np.pi / 2
    forward = circuit_to_differentiate(theta + shift)  # f(theta + pi/2)
    backward = circuit_to_differentiate(theta - shift)  # f(theta - pi/2)
    
    # Compute the gradient
    gradient = 0.5 * (forward - backward)
    return gradient


In [ ]:
# Define the built-in parameter-shift rule function
def parameter_shift_rule_built_in(circuit, theta):
    """Function that applies the PennyLane built-in parameter shift rule to a specific circuit with respect to the parameter theta.

    Args:
        circuit (qml.QNode): quantum circuit we want to differentiate.
        theta (float): parameter of the circuit with respect to which we differentiate.

    Returns:
        (np.tensor): a numpy tensor of 1 element corresponding to the derivative of the circuit with respect to theta.
    """  
    ##################
    # YOUR CODE HERE #
    ##################
    gradient_fn = qml.gradients.param_shift(circuit)
    gradient = gradient_fn(theta)
    return gradient
# Define the built-in classical_jacobian function
def jacobian_built_in(circuit, theta):
    """Function that applies the PennyLane built-in jacobian method to a specific circuit with respect to the parameter theta.

    Args:
        circuit (qml.QNode): quantum circuit we want to differentiate.
        theta (float): parameter of the circuit with respect to which we differentiate.

    Returns:
        (np.tensor): a numpy tensor of 1 element corresponding to the derivative of the circuit with respect to theta.
    """  
    ##################
    # YOUR CODE HERE #
    ################## 
    jacobian_fn = qml.jacobian(circuit)
    jacobian = jacobian_fn(theta)
    return jacobian


In [ ]:
def gradient_descent_optimization(quantum_circuit,initial_theta, learning_rate, max_iterations):
    """
    Performs Gradient Descent optimization to find the optimal parameter theta
    for a quantum circuit to minimize its output expectation value.

    Args:
        quantum_circuit (qml.QNode): A quantum circuit that depends of a parameter.
        initial_theta (np.array): An array with the initial value of the trainable parameter theta.
        learning_rate (float): Learning rate for the gradient descent update.
        max_iterations (int): Maximum number of iterations for the optimization.

    Returns:
        (np.array): A numpy array of 1 element corresponding to the optimized parameter theta.
    """
    ##################
    # YOUR CODE HERE #
    ##################
    theta = initial_theta
    for i in range(max_iterations):
        gradient = qml.grad(quantum_circuit)(theta)
        theta = theta - learning_rate * gradient
        
        print(f"Iteration {i+1}: theta = {theta}, f(theta) = {quantum_circuit(theta)}")

    return theta
    

optimized_theta=gradient_descent_optimization(circuit,np.array(0.1,requires_grad=True), 0.3, 50)
print(f"Optimized theta using Gradient Descent: {optimized_theta}")
print(f"Expectation value of quantum circuit at optimized theta: {circuit(optimized_theta)}")

In [ ]:
def gradient_descent_optimization_built_in(quantum_circuit,initial_theta, learning_rate, max_iterations):
    """
    Implements Gradient Descent optimization method of PennyLane to find the optimal parameter theta
    for a quantum circuit to minimize its output expectation value.

    Args:
        quantum_circuit (qml.QNode): A quantum circuit that depends of a parameter.
        initial_theta (np.array): An array with the initial value of the trainable parameter theta.
        learning_rate (float): Learning rate for the gradient descent update.
        max_iterations (int): Maximum number of iterations for the optimization.

    Returns:
        (np.array): A numpy array of 1 element corresponding to the optimized parameter theta.
    """
    ##################
    # YOUR CODE HERE #
    ##################
    opt = qml.GradientDescentOptimizer(stepsize= learning_rate)
    theta = initial_theta

    for i in range(max_iterations):
        theta = opt.step(quantum_circuit,theta)
        print(f"Iteration {i+1}: theta = {theta}, f(theta) = {quantum_circuit(theta)}")

    return theta

optimized_theta=gradient_descent_optimization_built_in(circuit,np.array(0.1,requires_grad=True), 0.3, 50)
print(f"Optimized theta using built-in Gradient Descent: {optimized_theta}")
print(f"Expectation value of quantum circuit at optimized theta: {circuit(optimized_theta)}")



In [ ]:
def cost_function(observable,params):
    """Computes the cost function we want to minimize.

    Args:
        observable (qml.Hamiltonian): a pennylane Hamiltonian whose expectation value we want to measure.
        params(np.array): an array with the trainable parameters of the ansatz.

    Returns:
        (np.tensor): a numpy tensor of 1 element corresponding to the cost function value
    """
    return strongly_entangling_ansatz(observable,params)

def optimizer(observable,params):
    """Updates the parameters to minimize the cost function value.

    Args:
        observable (qml.Hamiltonian): a pennylane Hamiltonian whose expectation value we want to measure.
        params(np.array): an array with the trainable parameters of the ansatz.

    Returns:
        (np.array): an array with the optimized trainable parameters.
    """
    def cost_fn(weights):
        return cost_function(observable, weights)

    max_steps = 100
    opt = qml.AdamOptimizer(0.1)  

    for _ in range(max_steps):
        # update the weights by one optimizer step
        params = opt.step(cost_fn, params)
    return params